#Nota informativa:
Este cuaderno sera utilizado para el desarrollo del proyecto final del certificado profesional "Ciencia de datos de IBM" disponible en la plataforma Coursera.

In [3]:
import pandas as pd
import numpy as np

In [4]:
print('¡Hola, Proyecto Final del Curso!')

¡Hola, Proyecto Final del Curso!
